The problem here is that I need to pass WKB to the get_walkability_data function.
The output of 'get_location' needs to look like this: 0103000020110F00000100000008000000E1BE8922A4F052C0CCA9CA4BFEA54340B4B4BCA59CEA52C0133BA84C31A54340C8597329AEE852C0B61850AA7D9E4340D96CB28009EB52C0101E9EA04DA0434079D8D47ABFED52C035C4E4D18D9E4340FFFA51103CEF52C0A831E49997A14340A87A1A2E72F052C0450FE55608A14340E1BE8922A4F052C0CCA9CA4BFEA54340



In [7]:
import logging
from shapely.geometry import Point
from shapely.wkb import dumps
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderUnavailable
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from pyproj import Transformer
import geopandas as gpd
from shapely import wkb
import streamlit as st
import math

# Retry configuration: 3 attempts with exponential backoff starting at 1 second
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=1, max=10), retry=retry_if_exception_type(GeocoderUnavailable))
def get_location(location_string, user_agent="location_walkability_app"):
    geolocator = Nominatim(user_agent=user_agent)
    location = geolocator.geocode(location_string, country_codes='us')

    if location:
        # Return coordinates in EPSG:4326 (longitude, latitude)
        return location.longitude, location.latitude

    logging.warning("Location not found")
    return None

def miles_to_degrees(miles, latitude):
    # Convert miles to degrees of latitude
    degrees_latitude = miles / 69.0
    
    # Convert miles to degrees of longitude
    degrees_longitude = miles / (69.0 * math.cos(math.radians(latitude)))
    
    return degrees_latitude, degrees_longitude

def get_walkability_data(location_string, buffer_radius_miles, conn):
    # Get the longitude and latitude from the get_location function
    longitude, latitude = get_location(location_string)

    # Convert miles to degrees
    degrees_latitude, degrees_longitude = miles_to_degrees(buffer_radius_miles, latitude)

    # Use the larger of the two degree values for the buffer radius
    buffer_radius_degrees = max(degrees_latitude, degrees_longitude)

    # Prepare the SQL query
    query = """
            SELECT geoid20, natwalkind, geometry
            FROM national_walkability_index
            WHERE ST_DWithin(
                st_setsrid(st_makepoint(:longitude, :latitude), 4326),
                geometry,
                :buffer_radius_degrees
            );
    """

    # Execute the query and fetch results
    df = conn.query(
        query,
        ttl="10m",  # Cache the result for 10 minutes
        params={"longitude": longitude, "latitude": latitude, "buffer_radius_degrees": buffer_radius_degrees}
    )

    # Convert DataFrame to GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=gpd.GeoSeries.from_wkb(df['geometry']))
    gdf.set_crs(epsg=4326, inplace=True)


    return gdf


In [8]:
import logging

# Configure logging
logging.basicConfig(level=logging.DEBUG)

conn = st.connection("postgresql", type="sql")
location_string = "Knoxville, TN"
buffer_radius_miles = 1.0

# Get coordinates
coordinates = get_location(location_string)
logging.debug(f'Coordinates for {location_string}: {coordinates}')

# Call the get_walkability_data function
try:
    walkability_data = get_walkability_data(location_string, buffer_radius_miles, conn)
    logging.debug(f'Walkability data: {walkability_data.head()}')
except Exception as e:
    logging.error(f'Error getting walkability data: {e}')

DEBUG:urllib3.util.retry:Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
DEBUG:urllib3.util.retry:Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)


DEBUG:geopy:Nominatim.geocode: https://nominatim.openstreetmap.org/search?q=Knoxville%2C+TN&format=json&limit=1&countrycodes=us
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): nominatim.openstreetmap.org:443
DEBUG:urllib3.connectionpool:https://nominatim.openstreetmap.org:443 "GET /search?q=Knoxville%2C+TN&format=json&limit=1&countrycodes=us HTTP/11" 200 467
DEBUG:root:Coordinates for Knoxville, TN: (-83.9210261, 35.9603948)
DEBUG:urllib3.util.retry:Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
DEBUG:urllib3.util.retry:Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
DEBUG:geopy:Nominatim.geocode: https://nominatim.openstreetmap.org/search?q=Knoxville%2C+TN&format=json&limit=1&countrycodes=us
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): nominatim.openstreetmap.org:443
DEBUG:urllib3.connectionpool:https://nominatim.openstreetmap.org:443 "GET /search?q=K

In [9]:
walkability_data.head()

,geoid20,natwalkind,geometry
0,470930066001,18.50,"POLYGON ((-83.93796 35.98145, -83.93131 35.985..."
1,470930070001,16.17,"POLYGON ((-83.94538 35.97442, -83.93503 35.976..."
2,470930069003,15.50,"POLYGON ((-83.93649 35.96418, -83.92968 35.968..."
3,470930069001,14.67,"POLYGON ((-83.93485 35.96098, -83.93067 35.962..."
4,470930009022,11.67,"POLYGON ((-83.93667 35.95218, -83.93281 35.954..."
